In [1]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_keyword")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])
df.head()

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,판례정보일련번호,선고,판결유형,...,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드
0,241209,법인세등부과처분취소[국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 ...,2022두51031,2024.07.25,대법원,세무,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,241209,선고,판결,...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,"[3] 대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...",【이 유】 1. 원고의 원심 승소 부분에 대한 상고의 적법 여부에 관한 직권...,{'이유': '1. 원고의 원심 승소 부분에 대한 상고의 적법 여부에 관한 직권판단...,승소 적법 승소 상소 두 취소 승소 여 로써 불복 존재 과세표준 증명책임 사업 연도...,0,None
1,240925,폭행·특수상해(인정된죄명:상해)·특수협박(인정된죄명:협박)·감금[피고인이 위험한 물...,2023도18812,2024.06.13,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,240925,선고,판결,...,"형법 제258조의2 제1항, 제257조 제1항, 제284조, 제283조 제1항은 위...","형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조","대법원 2002. 6. 14. 선고 2002도1341 판결, 대법원 2004. 6....",【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,【이 유】 상고이유를 판단한다. 1. 검사의 상고이유에 관하여 가. 2...,{'이유': '상고이유를 판단한다. 1. 검사의 상고이유에 관하여 가. 2019. ...,폭행 특수 특수 협박 주거지 입금 주기 형광등 값 화가 손 머리 수회 발 옆구리 주...,4,"['보험', '계약', '보험금', '지급', '자동차', '사고', '손해', '..."
2,240809,상해[형사소송법 제33조 제1항 제1호가 정한 필요적 국선변호인 선정사유인 ‘구속’...,2021도6357,2024.05.23,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,240809,선고,전원합의체 판결,...,[다수의견] 형사소송법 제33조 제1항 제1호는 피고인에게 변호인이 없는 때에 ...,"헌법 제12조, 형사소송법 제33조 제1항 제1호, 제69조, 제473조 제1항, ...","대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(...",【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,【이 유】 상고이유를 판단한다. 1. 사건의 개요와 쟁점 가. 소송의 진...,{'이유': '상고이유를 판단한다. 1. 사건의 개요와 쟁점 가. 소송의 진행 경과...,개요 쟁점 진행 경과 인천 건조물 침입 죄 제기 징역 구속영장 발부 건 언급 건조물...,0,None
3,219875,주거침입·성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영)·상해,2021노802,2021.11.05,울산지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,219875,선고,판결 : 상고,...,피고인이 별거 상태로 이혼소송 중이던 배우자 甲(女)이 거주하는 원룸에 베란다를...,성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항,None,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,【이 유】 1. 이 법원의 심판범위 가. 경합범으로 동시에 기소된 사건에 ...,{'이유': '1. 이 법원의 심판범위 가. 경합범으로 동시에 기소된 사건에 대하여...,경합 동시 기소 개일 분리 일부상소 쌍방 상소 분리 경합 지남 무죄판결 주거침입 벌...,4,"['보험', '계약', '보험금', '지급', '자동차', '사고', '손해', '..."
4,214727,마약류관리에관한법률위반(향정)ㆍ마약류관리에관한법률위반(대마)ㆍ특수상해,2019고단4056,2020.02.19,부산지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,214727,선고,판결,...,None,None,None,"【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...","【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...",【이 유】범 죄 사 실[범죄전력]피고인 1(2020. 6. 19.자 항소심 판...,{'이유': '범 죄 사 실[범죄전력]피고인 1(2020. 6. 19.자 항소심 판...,죄 사 실 전력 수원지방법원 폭력 감금 죄 징역 집행유예 중앙 마약 류 관리 대마 ...,4,"['보험', '계약', '보험금', '지급', '자동차', '사고', '손해', '..."


# 참조조문 전처리

In [2]:
df['참조조문']=df['참조조문'].fillna('참조조문 없음')


In [12]:
df['참조조문'].value_counts()

참조조문
참조조문 없음                                                                               803
민법 제750조                                                                               84
민법 제763조                                                                               68
자동차손해배상보장법 제3조                                                                         37
형사소송법 제308조                                                                            27
                                                                                     ... 
가.나.다.라.근로기준법 제27조 제1항 / 다. 구 근로기준법 제27조의2                                              1
가.        민법 제758조 제1항, 나.        제763조,         제393조,         상법 제727조                1
민법 제750조,         제763조,         제396조,         교통사고처리특례법 제3조,         민사소송법 제183조      1
형사소송법 제307조, 제308조                                                                      1
지방자치법 제102조,         국가배상법 제2조                                                          1
Name:

In [22]:
import re
def extract_laws(text):
    if isinstance(text, str):  # 텍스트가 문자열인지 확인
    # '법', '시행령', '에 관한 법률', '규칙'으로 끝나는 단어를 정규 표현식으로 추출
        laws = re.findall(r'\b(\w+(?:법|시행령|에 관한 법률|규칙))\b', text)
        return set(laws)
    return set()

# 각 참조조문에서 법과 시행령 이름을 찾기
law_names = set()
for reference in df['참조조문']:
    law_names.update(extract_laws(reference))


# 법과 시행령 이름을 붙이는 함수
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = None  # 이전 법 이름 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정
                    break

            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_names):
                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x  # 문자열이 아닐 경우 그대로 반환


In [23]:
law_names

{'가사소송법',
 '같은법',
 '같은법시행규칙',
 '같은법시행령',
 '거래신고에 관한 법률',
 '건널목개량촉진법',
 '건설기술관리법',
 '건설기술관리법시행령',
 '건설산업기본법',
 '건설산업기본법시행령',
 '건설업법',
 '건설업법시행령',
 '건축법',
 '건축법시행령',
 '검사규칙',
 '검찰집행사무규칙',
 '검찰청법',
 '게임산업진흥에 관한 법률',
 '경범죄처벌법',
 '경비업법',
 '경찰공무원법',
 '경찰관직무집행법',
 '경찰범처벌규칙',
 '고압가스안전관리법',
 '공동주택관리법',
 '공무원및사립학교교직원의료보험법',
 '공무원연금법',
 '공정거래에 관한 법률',
 '공직선거및선거부정방지법',
 '공직선거법',
 '관광진흥법',
 '관리에 관한 법률',
 '관세법',
 '광업법',
 '교육공무원법',
 '교육기본법',
 '교육법',
 '교통사고처리특례법',
 '구국가배상법',
 '구조개선에 관한 법률',
 '국가공무원법',
 '국가배상법',
 '국가배상법시행령',
 '국가보상법',
 '국가보안법',
 '국가소송법',
 '국가유공자등예우및지원에관한법률시행령',
 '국가재정법',
 '국민건강보험법',
 '국민의료보험법',
 '국민체육진흥법',
 '국바배상법',
 '국유철도건설규칙',
 '국유철도운전규칙',
 '국적법',
 '국제사법',
 '국회의원선거법',
 '군법회의법',
 '군사법원법',
 '군사원호보상급여금법',
 '군사원호보상법',
 '군수품관리법',
 '군수품관리법시행령',
 '군인사법',
 '군인연금법',
 '군형법',
 '귀속재산처리법',
 '규제에 관한 법률',
 '근로기준법',
 '근로기준법시행령',
 '근로복지기본법',
 '기본법',
 '난민법',
 '노동관계조정법',
 '노동위원회법',
 '노동쟁의조정법',
 '노동조합및노동관계조정법',
 '노동조합법',
 '노인복지법',
 '농어촌도로정비법',
 '농업협동조합법',
 '농지개혁법',
 '단속법',
 '대책에 관한 법률',
 '도로교통법',
 '도로교통

In [14]:
reason=df['참조조문'].apply(add_law_prefix)


0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1       형법 제257조 제1항, 형법 제258조의2 제1항, 형법 제283조 제1항, 형법...
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
3                            성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항
4                                                 참조조문 없음
                              ...                        
4184                                              참조조문 없음
4185                                              참조조문 없음
4186                                              참조조문 없음
4187               법원재난에 기인한 민형사사건 임시조치법 제1조, 군법회의법 제432조
4188                               지방자치법 제102조, 국가배상법 제2조
Name: 참조조문, Length: 4189, dtype: object

In [106]:
# 데이터프레임의 '참조조문' 열에 전처리 적용
df['참조조문'] = df['참조조문'].apply(add_law_prefix)

In [134]:
df['참조조문'][df['참조조문'].str.contains(', 제2항,')]

2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
6       형사소송법 제33조 제1항, 형사소송법 제361조의2 제1항, 제2항, 형사소송법 ...
101     민법 제396조, 민법 제750조, 민법 제751조, 민법 제763조, 병역법 제1...
165     도로교통법 제41조 제1항, 제2항, 도로교통법 제107조의2 제1호, 제2호, 도...
189     형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 형...
235     형법 제21조 제1항, 제2항, 형법 제22조 제1항, 제3항, 형법 제260조 제...
325     형법 제136조 제1항, 형사소송법 제211조, 형사소송법 제212조, 경찰관 직무...
384     헌법 제12조 제4항, 헌법 제27조, 형사소송법 제266조의3 제1항 제4호, 제...
394     교통사고처리특례법 제3조 제1항, 제2항, 형법 제268조, 도로교통법 제25조 제...
454     의료법 제53조 제1항, 제2항, 의료법 제59조 제1항, 행정소송법 제27조, 의...
568     교통사고처리 특례법 제4조 제1항, 교통사고처리 특례법 제1조, 특례법 제4조 제1...
598     국제사법 제2조 제1항, 제2항, 국제사법 제2조 제1항, 제2항, 민사소송법 제2...
604     행정소송법 제1조, 행정소송법 제2조, 행정소송법 제4조, 행정소송법 제19조, 행...
650         경찰관직무집행법 제1조, 경찰관직무집행법 제3조 제1항, 제2항, 제3항, 제7항
665     교통사고처리 특례법 제1조, 특례법 제4조 제1항, 제2항, 교통사고처리 특례법 제...
667     해상교통안전법시행규칙 제8조, 상법 제674조, 상법 제682조, 민법 제393조 ...
679     헌법 제13조 제1항, 형법 제1조 제1항, 성폭력범죄의 처벌 등에 관한 특례법 제...
713     헌법 제21

In [114]:
#조문 참조 정도 확인
refer_clause=[]
for clause in df['참조조문']:
    parts = clause.split(',')
    parts = [part.strip() for part in parts if part.strip()]
    for sub in parts:
        refer_clause.append(sub)
refer_clause = [item for item in refer_clause if item.strip()]
refcla=pd.DataFrame(data=refer_clause)
refcla.value_counts()

0               
참조조문 없음             803
민법 제763조            422
민법 제750조            391
민법 제393조            233
형사소송법 제308조         181
                   ... 
민사소송법 제193조 1항 4      1
민사소송법 제193조 2항        1
민사소송법 제198조           1
민사소송법 제201조           1
회사정리법 제96조            1
Name: count, Length: 3350, dtype: int64

In [113]:
df['참조조문'].value_counts()

참조조문
참조조문 없음                                               803
민법 제750조                                               86
민법 제763조                                               71
자동차손해배상보장법 제3조                                         37
민법 제763조, 민법 제393조                                     30
                                                     ... 
형법 제37조, 형법 제337조, 형사소송법 제298조                          1
형법 제335조, 형법 제333조                                      1
특정범죄가중처벌등에관한법률 제5조의5, 특정강력범죄의처벌에관한특례법 제3조, 형법 제35조      1
근로기준법 제33조, 근로기준법 제40조                                  1
지방자치법 제102조, 국가배상법 제2조                                  1
Name: count, Length: 2473, dtype: int64

In [123]:
# 리스트를 데이터프레임으로 변환
refcla = pd.DataFrame(data=refer_clause, columns=['조문'])

# 빈도 계산 결과를 데이터프레임으로 변환하여 모든 값 출력
refcla_counts_df = refcla.value_counts().reset_index(name='빈도')
refcla_counts_df.columns = ['조문', '빈도']


pd.set_option('display.max_rows',60)


refcla_counts_df  

,조문,빈도
0,참조조문 없음,803
1,민법 제763조,422
2,민법 제750조,391
3,민법 제393조,233
4,형사소송법 제308조,181
...,...,...
3345,민사소송법 제193조 1항 4,1
3346,민사소송법 제193조 2항,1
3347,민사소송법 제198조,1
3348,민사소송법 제201조,1


In [141]:
# 이전 법 이름, 조, 항 정보를 붙이는 함수
# 항과 호에 법, 조, 항을 붙이는 함수
def add_clause_prefix(parts):
    previous_law = None
    previous_clause = None
    previous_article = None
    updated_parts = []

    for part in parts:
        # 항으로 끝나는 경우
        if re.search(r'\d+항$', part):
            # 앞에 '법'과 '조'가 없는 경우
            if previous_clause and previous_law and not any(name in part for name in [previous_law, previous_clause] if name):
                part = f"{previous_law} {previous_clause} {part}"
            previous_article = part  # 항 정보 저장

        # 호로 끝나는 경우
        elif re.search(r'\d+호$', part):
            # 앞에 '법', '조', '항'이 모두 없는 경우
            if previous_clause and previous_article and previous_law and not any(name in part for name in [previous_law, previous_clause, previous_article] if name):
                part = f"{previous_law} {previous_clause} {previous_article} {part}"
            # 앞에 '항'만 있는 경우
            elif previous_clause and previous_law and previous_article and (previous_article in part) and not any(name in part for name in [previous_law, previous_clause] if name):
                part = f"{previous_law} {previous_clause} {part}"

        # 조로 끝나는 경우, 이전 법 이름과 조 정보 업데이트
        elif re.search(r'\d+조', part):
            previous_clause = part  # 조 정보 저장
            previous_article = None  # 항 정보 초기화

        # 법 이름이 있는 경우, previous_law 업데이트
        for law_name in law_names:
            if law_name in part:
                previous_law = law_name  
                break

        updated_parts.append(part)

    return updated_parts


# 2단계: 각 참조조문을 분리하고 항과 호에 앞의 정보 추가
refer_clause = []
for clause in df['참조조문']:
    parts = clause.split(',')
    parts = [part.strip() for part in parts if part.strip()]
    refer_clause.extend(add_clause_prefix(parts))

# 3단계: 분리된 조문을 데이터프레임에 저장하고 빈도 계산
refer_clause = [item for item in refer_clause if item.strip()]
refcla = pd.DataFrame(data=refer_clause, columns=['조문'])
refcla_counts = refcla.value_counts()
print(refcla_counts)

조문                                                                                                                                                                                                  
참조조문 없음                                                                                                                                                                                                 803
민법 제763조                                                                                                                                                                                                422
민법 제750조                                                                                                                                                                                                391
민법 제393조                                                                                                                                                                                       

내일시도

In [5]:
import re

# 법과 시행령 이름을 추출하는 함수
def extract_laws(text):
    if isinstance(text, str):  # 텍스트가 문자열인지 확인
        # '법' 또는 '시행령'으로 끝나는 단어를 정규 표현식으로 추출
        laws = re.findall(r'\b(\w+(?:법|시행령|에 관한 법률|규칙))(?=\s|$)', text)
        return set(laws)
    return set()

# 법과 시행령 이름을 수집
law_names = set()
for reference in df['참조조문']:
    law_names.update(extract_laws(reference))

# 법 이름과 조, 항을 붙이는 함수
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = None   # 이전 법 이름 저장
        previous_clause = None  # 이전 조 저장
        previous_article = None  # 이전 항 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name
                    break

            # 조로 끝나는 경우 법 이름과 조항을 저장
            if re.search(r'\d+조$', part):
                if previous_law and previous_law not in part:
                    part = f"{previous_law} {part}"  # 법 이름을 붙임
                previous_clause = part
                previous_article = None  # 항 정보 초기화

            # 항으로 끝나는 경우, 앞에 법과 조가 없는 경우 추가
            elif re.search(r'\d+항$', part):
                if previous_clause and previous_law and not any(name in part for name in [previous_law, previous_clause]):
                    part = f"{previous_law} {previous_clause} {part}"
                previous_article = part  # 항 정보 저장

            # 호로 끝나는 경우
            elif re.search(r'\d+호$', part):
                # 법, 조, 항이 모두 없는 경우 추가
                if previous_clause and previous_article and previous_law and not any(name in part for name in [previous_law, previous_clause, previous_article]):
                    part = f"{previous_law} {previous_clause} {previous_article} {part}"
                # 항만 있고 법과 조가 없는 경우
                elif previous_clause and previous_law and previous_article and (previous_article in part) and not any(name in part for name in [previous_law, previous_clause]):
                    part = f"{previous_law} {previous_clause} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x

# df에 함수 적용
clause= df['참조조문'].apply(add_law_prefix)
clause.value_counts()

참조조문
참조조문 없음                                               803
민법 제750조                                               86
민법 제763조                                               71
자동차손해배상보장법 제3조                                         37
민법 제763조, 민법 제393조                                     30
                                                     ... 
사회보호법 제5조 제1항 , 동법부칙 제2조, 형법 제12조                       1
형법 제37조, 제337조 ,형사소송법 제298조                             1
형법 제335조, 형법 제333조                                      1
특정범죄가중처벌등에관한법률 제5조의5, 특정강력범죄의처벌에관한특례법 제3조, 형법 제35조      1
지방자치법 제102조, 국가배상법 제2조                                  1
Name: count, Length: 2497, dtype: int64

In [105]:
pd.set_option('display.max_rows',None)


refcla_counts  

NameError: name 'refcla_counts' is not defined

# 참조조문 깔끔하게 구조 통일

1.',' 단위로 나눠지는 것과 [1]~~~ /[2]~~~ 이런식으로 나뉜게 있음.

2. [1]은 지우고 /[2] 이런식으로 되는건 ,로 대체하기.

3. 가.나.다.라. 와 같이 자음+'ㅏ'+'.'의 조합이 있으므로 이를 제외 '(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\\.'

In [75]:
import re


def clean_clause(text): # 전부 , 단위로 분리될수 있게 변환
    # 각 그룹을 '/'로 나누어 처리
    groups = text.split('/')
    results = []

    for group in groups:
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        group=re.sub(r'(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\.', '', group) # 가., 나., 다., 라., 마., 바., 사., 아., 자., 차., 카., 타., 파., 하. 형태만 제거
        group=re.sub(r'[\[\［]\d+[\]\］]\s*', '', group)  # '[1]', '[2]' 형식 제거
        group = group.strip()  # 각 그룹의 양쪽 공백 제거

        
        # 쉼표로 나눠 조각을 분리
        parts = re.split(r',\s*', group)
        transformed_parts = []

        for part in parts:
            part = part.strip()

            # 변환된 조문 추가
            transformed_parts.append(part)

        # 각 그룹을 쉼표로 연결하고 결과에 추가
        results.append(', '.join(transformed_parts))

    # 전체 결과를 반환
    return ', '.join(results)

# 예시 텍스트들에 대해 적용
examples = [
'가.나.라.        산업재해보상보험법 제3조 제1항 / 다.         민법 제756조'

]

for example in examples:
    print(clean_clause(example))

산업재해보상보험법 제3조 제1항, 민법 제756조


In [76]:
ccl=df['참조조문'].apply(clean_clause)
ccl

0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1             형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 제69조, 제473조 제1항, ...
3                            성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항
4                                                 참조조문 없음
                              ...                        
4184                                              참조조문 없음
4185                                              참조조문 없음
4186                                              참조조문 없음
4187               법원재난에 기인한 민형사사건 임시조치법 제1조, 군법회의법 제432조
4188                               지방자치법 제102조, 국가배상법 제2조
Name: 참조조문, Length: 4189, dtype: object

In [84]:
ccl[ccl.str.contains(r'현행 경찰관 직무집행법')]

3404    헌법 제37조 제2항, 구 경찰관 직무집행법(2014. 5. 20. 법률 제1260...
Name: 참조조문, dtype: object

In [80]:
df['참조조문'].iloc[3126]

'[1]         민사소송법 제248조[소의 제기],         구 근로기준법(2001. 8. 14. 법률 제6507호로 개정되기 전의 것) 제30조 제1항(현행         제23조 제1항 참조),         제33조 제1항(현행         제28조 제1항 참조),         제2항(현행 삭제),         구 노동조합 및 노동관계조정법 제84조 제1항(현행         근로기준법 제30조 제1항 참조),         제85조 제2항(현행         근로기준법 제31조 제2항 참조) / [2]         구 근로기준법(2001. 8. 14. 법률 제6507호로 개정되기 전의 것) 제30조 제1항(현행         제23조 제1항 참조) / [3]         구 근로기준법(2001. 8. 14. 법률 제6507호로 개정되기 전의 것) 제30조 제1항(현행         제23조 제1항 참조)      '

In [85]:
df.iloc[3404]

판례일련번호                                                      232579
사건명              손해배상(기)·손해배상(기)·손해배상(기)[국가가 노동조합 및 노동조합 지도부 등을...
사건번호                                                    2016다26662
선고일자                                                    2022.11.30
법원명                                                            대법원
사건종류명                                                           민사
판례상세링크           /DRF/lawService.do?OC=wanja1996&target=prec&ID...
판례정보일련번호                                                    232579
선고                                                              선고
판결유형                                                            판결
판시사항               [1] 경찰관이 불법적인 농성을 진압하는 과정에서 특정한 경찰장비를 필요한 최소...
판결요지               [1] 구 경찰관 직무집행법(2014. 5. 20. 법률 제12600호로 개정되...
참조조문              [1] 헌법 제37조 제2항, 구 경찰관 직무집행법(2014. 5. 20. 법률 ...
참조판례              [1] 대법원 2010. 11. 11. 선고 2010도7621 판결, 대법원 20...
판례내용             【원고, 피상고인 겸 상고인】 대한민국 (소송대리인 정부법무공단 담당변호사 조민현

In [112]:
# display.max_rows를 기본값으로 재설정
pd.reset_option('display.max_rows')

# display.max_columns를 기본값으로 재설정
pd.reset_option('display.max_columns')

# 참조조문 법이름 전처리

In [77]:
def extract_law_names(series):
    # 패턴 정의
    pattern = r'.*?(법|시행령|에 관한 법률|규칙)(\s*\([^()]*\))?'

    # 결과를 저장할 세트
    law_names_set = set()

    # 각 행에 대해 처리
    for text in series:
        # 텍스트를 ','로 분리하고 각 부분에 대해 strip() 적용
        parts = text.split(',')
        for part in parts:
            part = part.strip()
            # 정규 표현식으로 매칭
            match = re.match(pattern, part)
            if match:
                # 매칭된 내용을 세트에 추가
                law_names_set.add(match.group(0).strip())

    # 세트로 중복 제거 후 리스트로 반환
    return list(law_names_set)

law_names = extract_law_names(ccl)

In [78]:
law_names

['병역법',
 '구 노동조합 및 노동관계조정법',
 '해사안전법',
 '구 노동위원회법(2007. 1. 26. 법률 제8296호로 개정되기 전의 것)',
 '지방자치법',
 '구 폭력행위 등 처벌에 관한 법률(2006. 3. 24. 법률 제7891호로 개정되기 전의 것)',
 '구 출입국관리법',
 '폭력행위등처벌에 관한 법률',
 '구 경찰관 직무집행법(2014. 5. 20. 법률 제12600호로 개정되기 전의 것)',
 '4·16세월호참사 피해구제 및 지원 등을 위한 특별법',
 '구 장애인차별금지 및 권리구제 등에 관한 법률(2017. 12. 19. 법률 제15272호로 개정되기 전의 것)',
 '실화책임에관한법',
 '제조물 책임법',
 '산업재해보상보험법',
 '마약류관리에 관한 법률',
 '변호사법',
 '국유철도건설규칙',
 '구 교통약자의 이동편의 증진법',
 '구 액화석유가스의안전및사업관리법',
 '폭력행위등처벌에관한법',
 '경범죄처벌법',
 '자유형 등에 관한 검찰집행사무규칙',
 '구 자동차운송사업법',
 '총포·도검·화약류등단속법',
 '구 아동복지법(2017. 10. 24. 법률 제14925호로 개정되기 전의 것)',
 '어린이제품 안전 특별법',
 '게임산업진흥에 관한 법률',
 '공직선거및선거부정방지법',
 '구 섭외사법(2001. 4. 7. 법률 제6465호 국제사법으로 전문 개정되기 전의 것)',
 '소방기본법',
 '구 의료법(2006. 10. 27. 법률 제8067호로 개정되기 전의 것)',
 '구 건설업법(1994. 1. 7. 법률 제4724호로 개정되기 전의 것)',
 '구 국민건강보험법(2008. 3. 28. 법률 제9022호로 개정되기 전의 것)',
 '선박법',
 '지방재정법',
 '구 상가건물 임대차보호법(2009. 1. 30. 법률 제9361호로 개정되기 전의 것)',
 '특정경제범죄가중처벌등에관한법',
 '난민법',
 '총포·도검·화약류단속법',
 '도로법',
 '고용보험 및 산업재해보상보험의 보험료징수 등에

In [9]:
def extract_law_names(text):
    # 법 이름을 찾기 위한 정규식 패턴 정의
    law_pattern = r'^(.*?)\b(법|시행령|에 관한 법률|규칙|시행규칙)\b'

    # 각 법 이름을 저장할 리스트
    law_names = []

    # ','로 분할하여 각 조각에서 법 이름을 찾음
    parts = text.split(',')
    for part in parts:
        part = part.strip()  # 양쪽 공백 제거
        match = re.match(law_pattern, part)
        if match:
            law_names.append(match.group(0))  # 전체 매칭된 부분을 추가

    return law_names
text = "산업재해보상보험법 제11조, 민법 제763조, 구 산업재해보상보험법 시행규칙 제1조, 근로기준법 시행령 제10조, 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것)"
print(extract_law_names(text))
law_names1 = set()
for reference in ccl:
    law_names1.update(extract_laws(reference))
law_names1

['구 산업재해보상보험법 시행규칙', '근로기준법 시행령']


NameError: name 'extract_laws' is not defined

In [13]:
def extract_law_names(text):
    # 법 이름을 찾기 위한 정규식 패턴 정의 (괄호가 있으면 포함하여 추출)
    law_pattern = r'(?:\S+\s*)*?(?:법|시행령|에 관한 법률|규칙)(?:\s*\(.*?\))?'

    # 각 법 이름을 저장할 리스트
    law_names = []

    # ','로 분할하여 각 조각에서 법 이름을 찾음
    parts = text.split(',')
    for part in parts:
        part = part.strip()  # 양쪽 공백 제거
        print(part)
        match = re.match(law_pattern, part)
        print(match)
        if match:
            law_names.append(match.group(0))  # 전체 매칭된 부분을 추가

    return law_names
law_pattern = r'(\S*?(법|시행령|에 관한 법률|규칙)(?:\s*\(.*?\))?)'
# 예시 테스트
text = "산업재해보상보험법 제11조, 민법 제763조, 구 산업재해보상보험법 시행규칙 제1조, 근로기준법 시행령 제10조, 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제 1조"
matches = re.finditer(law_pattern, text)

for match in matches:
    print(match.group(0).strip())
# print(extract_law_names(text))

산업재해보상보험법
민법
산업재해보상보험법
시행규칙
근로기준법
시행령
법
인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것)


In [34]:
pattern=r'.*(법|시행령|에 관한 법률|규칙)(\s*\([^()]*\))?'
parts=text.split(',')
for part in parts:
    part=part.strip()
    print(part)
    match = re.match(pattern, part)
    print(match)

산업재해보상보험법 제11조
<re.Match object; span=(0, 9), match='산업재해보상보험법'>
민법 제763조
<re.Match object; span=(0, 2), match='민법'>
구 산업재해보상보험법 시행규칙 제1조
<re.Match object; span=(0, 16), match='구 산업재해보상보험법 시행규칙'>
근로기준법 시행령 제10조
<re.Match object; span=(0, 9), match='근로기준법 시행령'>
법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제 1조
<re.Match object; span=(0, 20), match='법인세법(2006. 12. 30. 법'>


In [89]:
def extract_law_names(series):
    # 괄호 외부에서 "법", "시행령", "에 관한 법률", "규칙"이 있는지 찾는 패턴
    outer_pattern = r'.*?(법|시행령|에 관한 법률|규칙)(\s*\([^()]*\))?'
    
    # 결과를 저장할 세트
    law_names_set = set()

    # 각 행에 대해 처리
    for text in series:
        # 텍스트를 ','로 분리하고 각 부분에 대해 strip() 적용
        parts = text.split(',')
        for part in parts:
            part = part.strip()
            
            # 괄호 바깥에서 패턴 매칭
            outer_match = re.match(outer_pattern, part)
            if outer_match:
                # 괄호 안에 "법", "시행령", "에 관한 법률", "규칙"이 포함되어 있으면 무시
                inner_pattern = r'\(([^)]*?\b(?:법|시행령|에 관한 법률|규칙)\b.*?)\)'
                if not re.search(inner_pattern, part):
                    # 매칭된 내용을 세트에 추가
                    law_names_set.add(outer_match.group(0).strip())

    # 세트로 중복 제거 후 리스트로 반환
    return list(law_names_set)

law_names2 = extract_law_names(ccl)

In [90]:
law_names2

['병역법',
 '구 노동조합 및 노동관계조정법',
 '해사안전법',
 '구 노동위원회법(2007. 1. 26. 법률 제8296호로 개정되기 전의 것)',
 '지방자치법',
 '구 폭력행위 등 처벌에 관한 법률(2006. 3. 24. 법률 제7891호로 개정되기 전의 것)',
 '구 출입국관리법',
 '폭력행위등처벌에 관한 법률',
 '구 경찰관 직무집행법(2014. 5. 20. 법률 제12600호로 개정되기 전의 것)',
 '4·16세월호참사 피해구제 및 지원 등을 위한 특별법',
 '구 장애인차별금지 및 권리구제 등에 관한 법률(2017. 12. 19. 법률 제15272호로 개정되기 전의 것)',
 '실화책임에관한법',
 '제조물 책임법',
 '산업재해보상보험법',
 '마약류관리에 관한 법률',
 '변호사법',
 '국유철도건설규칙',
 '구 교통약자의 이동편의 증진법',
 '구 액화석유가스의안전및사업관리법',
 '폭력행위등처벌에관한법',
 '경범죄처벌법',
 '자유형 등에 관한 검찰집행사무규칙',
 '구 자동차운송사업법',
 '총포·도검·화약류등단속법',
 '구 아동복지법(2017. 10. 24. 법률 제14925호로 개정되기 전의 것)',
 '어린이제품 안전 특별법',
 '게임산업진흥에 관한 법률',
 '공직선거및선거부정방지법',
 '구 섭외사법(2001. 4. 7. 법률 제6465호 국제사법으로 전문 개정되기 전의 것)',
 '소방기본법',
 '구 의료법(2006. 10. 27. 법률 제8067호로 개정되기 전의 것)',
 '구 건설업법(1994. 1. 7. 법률 제4724호로 개정되기 전의 것)',
 '구 국민건강보험법(2008. 3. 28. 법률 제9022호로 개정되기 전의 것)',
 '선박법',
 '지방재정법',
 '구 상가건물 임대차보호법(2009. 1. 30. 법률 제9361호로 개정되기 전의 것)',
 '특정경제범죄가중처벌등에관한법',
 '난민법',
 '총포·도검·화약류단속법',
 '도로법',
 '고용보험 및 산업재해보상보험의 보험료징수 등에

# 241031 마지막. -> 이거 매치하는거에서 match.group(2)에서 (괄호가 있는거) 없을때의 경우 pattern과 일치하는게 있을때만 법 추가하는걸로

In [ ]:
def extract_law_names(series):
    # 패턴 정의: "법", "시행령", "에 관한 법률", "규칙"으로 끝나는 패턴 + 괄호 포함 여부 처리
    law_pattern = r'.*?(법|시행령|에 관한 법률|규칙)(\s*\([^()]*\))?'
    pattern=r'.*?(법|시행령|에 관한 법률|규칙)'
    # 결과를 저장할 세트
    law_names_set = set()

    # 각 행에 대해 처리
    for text in series:
        # 텍스트를 ','로 분리하고 각 부분에 대해 strip() 적용

        parts = text.split(',')

        for part in parts:
            part = part.strip()

            # 패턴 매칭
            match = re.match(law_pattern, part)
            print(match)
            if match:
                # 괄호 포함하여 패턴이 매칭된 경우 전체를 포함해 법 이름으로 인식
                print('match.group(0): ',match.group(0))
                print('match.group(2): ',match.group(2))
                if match.group(2):  # 괄호가 있는 경우
                    law_name = match.group(0).strip()
                    print(law_name)
                else:  # 괄호가 없는 경우
                    match_
                    law_name = match.group(1).strip()
                # 세트에 추가
                law_names_set.add(law_name)

    # 세트로 중복 제거 후 리스트로 반환
    return list(law_names_set)

# 예시 실행
text = ["구 노동조합 및 노동관계조정법 제84조 제1항(현행 근로기준법 제30조 제1항 참조), 제85조 제2항(현행 근로기준법 제31조 제2항 참조), 형사소송법(2005. 5. 31. 법률 제7545호로 개정되기 전의 것) 제 3조 제2항"]
law_names = extract_law_names(text)
print(law_names)

<re.Match object; span=(0, 16), match='구 노동조합 및 노동관계조정법'>
match.group(0):  구 노동조합 및 노동관계조정법
match.group(2):  None
<re.Match object; span=(0, 17), match='제85조 제2항(현행 근로기준법'>
match.group(0):  제85조 제2항(현행 근로기준법
match.group(2):  None
<re.Match object; span=(0, 40), match='형사소송법(2005. 5. 31. 법률 제7545호로 개정되기 전의 것)'>
match.group(0):  형사소송법(2005. 5. 31. 법률 제7545호로 개정되기 전의 것)
match.group(2):  (2005. 5. 31. 법률 제7545호로 개정되기 전의 것)
형사소송법(2005. 5. 31. 법률 제7545호로 개정되기 전의 것)
['형사소송법(2005. 5. 31. 법률 제7545호로 개정되기 전의 것)', '법']


In [ ]:
def extract_law_names(series):
    # 패턴 정의: "법", "시행령", "에 관한 법률", "규칙"으로 끝나는 패턴 + 괄호 포함 여부 처리
    law_pattern = r'.*?(법|시행령|에 관한 법률|규칙)(\s*\([^()]*\))?'
    
    # 결과를 저장할 세트
    law_names_set = set()

    # 각 행에 대해 처리
    for text in series:
        # 텍스트를 ','로 분리하고 각 부분에 대해 strip() 적용
        parts = text.split(',')
        for part in parts:
            part = part.strip()
            
            # 패턴 매칭
            match = re.match(law_pattern, part)
            if match:
                # 괄호 포함하여 패턴이 매칭된 경우 전체를 포함해 법 이름으로 인식
                if match.group(3):  # 괄호가 있는 경우
                    law_name = match.group(1).strip() + match.group(3).strip()
                else:  # 괄호가 없는 경우
                    law_name = match.group(1).strip()
                # 세트에 추가
                law_names_set.add(law_name)

    # 세트로 중복 제거 후 리스트로 반환
    return list(law_names_set)

# 참조판례 전처리

In [87]:
df['참조판례']=df['참조판례'].fillna('참조판례 없음')
#공백은 '참조판례 없음'으로 대체함.

In [94]:
#참조판례 정리

clean_part=[]
for part in df['참조판례']:
    part = part.strip()
    part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)  # 대괄호와 그 안의 내용 제거
    part = re.sub(r'\s*[가-힣]\.\s*', '', part)  # '가.', '나.', '다.'와 같은 패턴 제거
    part = part.strip('<br/>')  # 공백 제거
    part = part.replace('/', ',')  # '/'를 ','로 변경
    part = re.sub(r'\s+', ' ', part).strip()
    clean_part.append(part)

df['참조판례']=clean_part




In [95]:
df['참조판례']

0       대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2312),...
1       대법원 2002. 6. 14. 선고 2002도1341 판결, 대법원 2004. 6....
2       대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(...
3                                                 참조판례 없음
4                                                 참조판례 없음
                              ...                        
4184                                              참조판례 없음
4185                                              참조판례 없음
4186                                              참조판례 없음
4187                                              참조판례 없음
4188    1966.1.25. 선고 65다2257 판결(판례카아드 1458호, 대법원판결집 1...
Name: 참조판례, Length: 4189, dtype: object

In [99]:
#판례 참조 정도 확인

refered_jud=[]
for clean_part in df['참조판례']:
    parts = re.split(r',\s*(?![^()]*\))', clean_part)
    for sub in parts:
        refered_jud.append(sub)

refered_jud = [item for item in refered_jud if item.strip()]

refjud=pd.DataFrame(data=refered_jud)
refjud.value_counts()

0                                                                          
참조판례 없음                                                                        2281
대법원 2000. 7. 4. 선고 99도4341 판결(공2000하, 1851)                                       9
대법원 1994. 9. 13. 선고 94도1335 판결(공1994하, 2695)                                      8
대법원 2002. 6. 28. 선고 2002도2001 판결(공2002하, 1893)                                    8
대법원 1995. 5. 26. 선고 94다60509 판결(공1995하, 2258)                                     7
                                                                               ... 
대법원 1983.12.27. 선고 83도2557 판결                                                     1
대법원 1983.12.27. 선고 83다카644 판결(공1984,259)                                          1
대법원 1983.12.27 선고 83다카1503 판결                                                     1
대법원 1983.12.13. 선고 83다카975 판결(요특Ⅱ 자동차손해배상보장법 제3조(82)1222면 집31⑥민71 공721호163)       1
형사소송법 제310조(8)1452면)                                                              1


['대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2312)',
 '대법원 2021. 2. 25. 선고 2017두237 판결',
 '대법원 2024. 4. 12. 선고 2023두58701 판결(공2024상, 800)',
 '대법원 2002. 6. 14. 선고 2002도1341 판결',
 '대법원 2004. 6. 11. 선고 2004도2018 판결(공2004하, 1198)',
 '대법원 2017. 3. 30. 선고 2017도771 판결',
 '대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(변경)',
 '대법원 2011. 9. 29. 선고 2011도10441 판결(변경)',
 '대법원 2011. 11. 10. 선고 2011도12108 판결(변경)',
 '대법원 2012. 6. 28. 선고 2012도5420 판결(변경)',
 '대법원 2016. 6. 9. 선고 2016도4479 판결(변경)',
 '대법원 2017. 1. 12. 선고 2016도19006 판결(변경)',
 '참조판례 없음',
 '참조판례 없음',
 '참조판례 없음',
 '대법원 2010. 5. 27. 선고 2010도3377 판결(공2010하, 1328) ',
 '대법원 1973. 9. 12. 선고 73도1919 판결',
 '대법원 1973. 10. 10. 선고 73도2142 판결',
 '대법원 2011. 2. 10. 선고 2008도4558 판결(공2011상, 593)',
 '참조판례 없음',
 '대법원 2000. 3. 10. 선고 2000도159 판결(공2000상, 1001)',
 '대법원 2004. 4. 27. 선고 2004도482 판결(공2004상, 946)',
 '대법원 2006. 4. 14. 선고 2005도9561 판결(2006상, 836)',
 '참조판례 없음',
 '1970.7.28. 선고 70도1358 판결(요형 형법 제10조(26)16면카9082 집18②형75)',
 '1980.5.27. 선고 80도656 판결(요형 사회보호법 